# House Prices: Advanced Regression Techniques
Ensemble approach: XGBoost, Ridge Regression, LightGBM

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.ensemble import StackingRegressor
from scipy.stats import skew
import logging
logging.getLogger('lightgbm').setLevel(logging.ERROR)

## 1. Data Loading

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
test_ID = test['Id']  # Save test IDs for submission

## 2. Preprocessing and Feature Engineering

### 2.1 Log-transform Outputs (SalesPrice)

Using a logarithmic transformation can be very helpful for several reasons:

1. Reducing Skewness:
Housing prices are often right-skewed, meaning there are a few extremely high values compared to most homes. Taking the logarithm of SalePrice tends to produce a distribution that is more symmetric and closer to normal. Many modeling techniques work better when the target variable is normally distributed.

2. Stabilizing Variance:
The log transformation helps reduce heteroscedasticity, where the variability of the errors changes with the magnitude of the target variable. With a log-transformed target, the variance tends to be more constant across different levels, making the model’s assumptions more valid.

3. Relative Error Emphasis:
When you predict in the log space, differences are interpreted in relative rather than absolute terms. This means that a given percentage error is treated consistently, regardless of whether the sale price is high or low. For example, a 10% error on a \\$100,000 home is treated similarly to a 10\% error on a \\$1,000,000 home.

4. Mitigating the Impact of Outliers:
Since the log transform compresses the range of high values, it reduces the influence of extreme outliers. This can lead to more robust models that aren’t overly swayed by a few high-priced properties.

Note: `np.log1p` is used because it can handle zero values and is defined for all non-negative numbers, ensuring numerical stability.

In [3]:
# Log-transform the target for a better distribution
train["SalePrice"] = np.log1p(train["SalePrice"])

# Combine train and test data for uniform processing
ntrain = train.shape[0]
all_data = pd.concat((train.drop('SalePrice', axis=1), test)).reset_index(drop=True)

# Fill missing values:
# - Numeric: fill with median
# - Categorical: fill with "None"
numeric_feats = all_data.select_dtypes(include=[np.number]).columns
for col in numeric_feats:
    all_data[col].fillna(all_data[col].median(), inplace=True)
categorical_feats = all_data.select_dtypes(include=[object]).columns
for col in categorical_feats:
    all_data[col].fillna("None", inplace=True)

### 2.2 Transform Skewed Numeric Features

In the preprocessing step, we identify numeric features with high skewness (absolute skew greater than 0.75) because highly skewed data can negatively impact the performance of many machine learning models. Skewed data often has outliers or long tails, causing models to be overly sensitive to extreme values.

To reduce skewness, we apply a log transformation (`np.log1p`), which converts the distribution closer to a normal distribution. Specifically:

- First, we calculate skewness for each numeric feature using `scipy.stats.skew`.
- Features with absolute skewness greater than 0.75 are considered highly skewed and targeted for transformation.

This transformation makes the feature distributions more symmetrical and improves model stability and performance.


In [4]:
# Transform skewed numeric features (absolute skew > 0.75)
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna()))
skewed_features = skewed_feats[abs(skewed_feats) > 0.75].index
print("Skewed features:", list(skewed_features))
for feat in skewed_features:
    all_data[feat] = np.log1p(all_data[feat])

# One-hot encode categorical features
all_data = pd.get_dummies(all_data)

# Split the data back into training and test sets
train_data = all_data[:ntrain]
test_data = all_data[ntrain:]
y = train["SalePrice"]

Skewed features: ['MSSubClass', 'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtHalfBath', 'KitchenAbvGr', 'TotRmsAbvGrd', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']


## 3. Train / Holdout Split (20% Holdout)

In [5]:
X_train, X_holdout, y_train, y_holdout = train_test_split(train_data, y, test_size=0.2, random_state=42)

## 4. Grid Searches

### 4.1 XGBoost

XGBoost (eXtreme Gradient Boosting) is a powerful and scalable gradient boosting framework that has become a popular tool for regression tasks. It is widely recognized for its speed, performance, and flexibility in handling various data challenges.

#### Overview

XGBoost builds an ensemble of weak prediction models, typically decision trees, in a sequential manner. Each new tree is added to correct the errors made by the previous trees. The final prediction is a sum of the predictions from all individual trees.

#### How It Works

The model predicts a target value $\hat{y}$ using an additive model:

$$
\hat{y} = \sum_{k=1}^{K} f_k(x),
$$

where:
- $f_k(x)$ is a function representing the $k^\text{th}$ decision tree,
- $K$ is the total number of trees in the ensemble.

#### Objective Function

The objective function of XGBoost is designed to balance the training loss with a regularization term to control model complexity. It is given by:

$$
\mathcal{L}(\phi) = \sum_{i=1}^{n} \ell\left(y_i, \hat{y}_i\right) + \sum_{k=1}^{K} \Omega(f_k),
$$

where:
- $\ell(y_i, \hat{y}_i)$ is the loss function for the $i^\text{th}$ sample (for regression, typically $\ell(y, \hat{y}) = \left(y - \hat{y}\right)^2$),
- $\Omega(f_k)$ is the regularization term for the $k^\text{th}$ tree, defined as:

$$
\Omega(f) = \gamma T + \frac{1}{2} \lambda \|w\|^2,
$$

with:
- $T$ representing the number of leaves in the tree,
- $w$ being the vector of scores on leaves,
- $\gamma$ and $\lambda$ as regularization parameters controlling the model complexity.

#### Advantages

- **High Performance:** Optimized for speed and efficiency, XGBoost often outperforms many other regression algorithms.
- **Regularization:** Built-in regularization techniques help in reducing overfitting.
- **Scalability:** Supports parallel processing and distributed computing, making it effective on large datasets.
- **Flexibility:** Offers a variety of hyperparameters and supports custom objective functions, allowing for fine-tuning and adaptability to different problems.

#### Comparison with Other Methods

- **LightGBM:** While LightGBM uses a leaf-wise growth strategy that can be faster on large datasets, XGBoost is known for its robustness and flexibility.
- **Random Forest:** Unlike Random Forest, where trees are built independently, XGBoost builds trees sequentially, enabling it to correct previous errors and often yield improved predictive performance.

#### Practical Considerations

- **Hyperparameter Tuning:** Key parameters include the number of trees, learning rate, maximum depth, and regularization terms. Cross-validation is essential for finding the optimal settings.
- **Feature Importance:** XGBoost provides various methods to extract feature importance, which can be critical for model interpretation.
- **Early Stopping:** Use early stopping based on a validation set to avoid overfitting.
- **Handling Missing Values:** XGBoost is designed to handle missing values efficiently, which can simplify preprocessing.

XGBoost's balance of computational efficiency and predictive power makes it an excellent choice for tackling complex regression problems in both research and production environments.


#### Why is XGBoost Effective for Regression on Tabular Data?

1. **Regularization and Robustness:**  
   XGBoost incorporates both L1 and L2 regularization, which helps manage overfitting—a common issue in tabular datasets with noisy features.

2. **Handling Missing Values:**  
   The algorithm can automatically learn the best way to handle missing data, which is often encountered in real-world tabular datasets.

3. **Gradient Boosting Mechanism:**  
   By iteratively fitting new models to the residual errors of previous models, XGBoost effectively minimizes the loss. At each iteration \( t \), the model minimizes an approximated objective:

   $$
   \mathcal{L}^{(t)} \approx \sum_{i=1}^{n} \left[ g_i f_t(x_i) + \frac{1}{2} h_i f_t(x_i)^2 \right] + \Omega(f_t)
   $$

   where $  g_i = \frac{\partial \ell(y_i, \hat{y}_i^{(t-1)})}{\partial \hat{y}_i^{(t-1)}}  $ is the gradient, and $  h_i = \frac{\partial^2 \ell(y_i, \hat{y}_i^{(t-1)})}{\partial \hat{y}_i^{(t-1)2}}  $ is the Hessian (second derivative).

4. **Efficient Computation:**  
   XGBoost is designed for parallel processing, which speeds up the training process, especially on large tabular datasets.

5. **Capturing Feature Interactions:**  
   The tree-based model naturally captures non-linear relationships and interactions between features, making it very effective when these interactions are important for predicting the target variable.

6. **Customizable Loss Functions:**  
   Users can tailor the loss function to better suit the specific needs of the regression task, enhancing model performance.

In summary, XGBoost's robust regularization, efficient handling of missing data, gradient boosting mechanism, and ability to model complex interactions make it particularly effective for regression tasks on tabular data.


In [6]:
param_grid_xgb = {
    "max_depth": [3, 4],
    "learning_rate": [0.01, 0.05],
    "n_estimators": [100, 200],
    "reg_alpha": [0, 0.1, 1],
    "reg_lambda": [1, 1.5],
    "subsample": [0.8],
    "colsample_bytree": [0.8]
}

xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1
)

grid_search_xgb = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_xgb,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1
)
grid_search_xgb.fit(X_train, y_train)
print("Best XGBoost parameters:", grid_search_xgb.best_params_)
best_xgb = grid_search_xgb.best_estimator_

holdout_preds_xgb = best_xgb.predict(X_holdout)
rmse_xgb = np.sqrt(mean_squared_error(np.expm1(y_holdout), np.expm1(holdout_preds_xgb)))
print("Holdout RMSE (XGBoost):", rmse_xgb)

Fitting 3 folds for each of 48 candidates, totalling 144 fits
Best XGBoost parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 200, 'reg_alpha': 0, 'reg_lambda': 1, 'subsample': 0.8}
Holdout RMSE (XGBoost): 25268.083955857885


### 4.2 Ridge Regression

Ridge Regression is a regularized version of Ordinary Least Squares (OLS) regression. It extends the standard OLS method by adding an L2 penalty term, which helps to prevent overfitting and address issues like multicollinearity.

#### Ordinary Least Squares (OLS)

In OLS, the goal is to minimize the residual sum of squares (RSS) to estimate the regression coefficients. The OLS objective function is given by:

$$
\min_{\beta} \| y - X\beta \|_2^2 = \min_{\beta} \sum_{i=1}^{n} \left( y_i - \sum_{j=1}^{p} X_{ij}\beta_j \right)^2,
$$

where:
- $X \in \mathbb{R}^{n \times p}$ is the design matrix,
- $y \in \mathbb{R}^{n}$ is the response vector,
- $\beta \in \mathbb{R}^{p}$ is the vector of regression coefficients.

#### Ridge Regression Formulation

Ridge regression modifies the OLS objective by adding an L2 regularization term. The ridge regression objective function becomes:

$$
\min_{\beta} \left\{ \| y - X\beta \|_2^2 + \lambda \|\beta\|_2^2 \right\},
$$

where:
- $\|\beta\|_2^2 = \sum_{j=1}^{p} \beta_j^2$ is the squared L2 norm of the coefficient vector,
- $\lambda \geq 0$ is the regularization parameter that controls the amount of shrinkage applied to the coefficients.

This additional term penalizes large coefficient values, effectively shrinking them towards zero and reducing model complexity.

#### Closed-Form Solution

The solution for the ridge regression coefficients can be derived by taking the derivative of the objective function with respect to $\beta$ and setting it to zero. The closed-form solution is:

$$
\hat{\beta} = \left( X^T X + \lambda I \right)^{-1} X^T y,
$$

where $I$ is the $p \times p$ identity matrix. Notice that if $\lambda = 0$, the solution reverts to the standard OLS estimator.

#### Key Considerations

- **Regularization Effect:** The parameter $\lambda$ controls the strength of the penalty. A larger $\lambda$ increases the bias in the estimates but can reduce variance, which may improve predictive performance on new data.
- **Multicollinearity:** Ridge regression is particularly useful when predictor variables are highly correlated, as the regularization term stabilizes the estimation process.
- **Feature Scaling:** Since the penalty term is sensitive to the scale of the predictors, it is important to standardize or normalize the features before applying ridge regression.


In [7]:
ridge = Ridge()
param_grid_ridge = {"alpha": [0.1, 1.0, 10.0]}

grid_search_ridge = GridSearchCV(
    estimator=ridge,
    param_grid=param_grid_ridge,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1
)
grid_search_ridge.fit(X_train, y_train)
print("Best Ridge parameters:", grid_search_ridge.best_params_)
best_ridge = grid_search_ridge.best_estimator_

holdout_preds_ridge = best_ridge.predict(X_holdout)
rmse_ridge = np.sqrt(mean_squared_error(np.expm1(y_holdout), np.expm1(holdout_preds_ridge)))
print("Holdout RMSE (Ridge):", rmse_ridge)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Best Ridge parameters: {'alpha': 10.0}
Holdout RMSE (Ridge): 26133.69519708773


### 4.3 LightGBM

LightGBM (Light Gradient Boosting Machine) is a high-performance, gradient boosting framework based on decision tree algorithms. It is designed to be distributed, efficient, and scalable, making it an excellent choice for large-scale regression tasks.

#### Overview

LightGBM builds an ensemble of decision trees in a boosting framework. Unlike traditional tree-based methods, it grows trees in a leaf-wise (best-first) manner rather than level-wise, which can lead to faster convergence and often higher accuracy. This approach, combined with histogram-based algorithms, helps reduce memory usage and speeds up training.

#### How It Works

In LightGBM, the final prediction is an additive model composed of multiple decision trees:

$$
\hat{y} = \sum_{k=1}^{K} f_k(x),
$$

where:
- $\hat{y}$ is the predicted value,
- $f_k(x)$ represents the prediction from the $k^\text{th}$ tree,
- $K$ is the total number of trees.

Each tree is built to minimize the residual errors of the previous trees in the sequence.

#### Objective Function

For regression tasks, LightGBM typically minimizes a loss function such as the Mean Squared Error (MSE), combined with a regularization term to avoid overfitting. The objective function can be expressed as:

$$
\mathcal{L} = \sum_{i=1}^{n} \ell\left(y_i, \hat{y}_i\right) + \sum_{k=1}^{K} \Omega\left(f_k\right),
$$

where:
- $\ell\left(y_i, \hat{y}_i\right)$ is the loss for the $i^\text{th}$ sample, commonly $\ell(y, \hat{y}) = \left(y - \hat{y}\right)^2$,
- $\Omega\left(f_k\right)$ is the regularization term for the $k^\text{th}$ tree,
- $n$ is the total number of samples.

#### Advantages

- **Speed and Efficiency:** Utilizes a histogram-based algorithm to speed up training and reduce memory usage.
- **Scalability:** Can efficiently handle large datasets and high-dimensional data.
- **Accuracy:** The leaf-wise growth strategy often results in more accurate models compared to level-wise methods.
- **Flexibility:** Supports parallel training and GPU acceleration, making it suitable for intensive computational tasks.

#### Comparison with Other Methods

- **XGBoost:** While both are gradient boosting frameworks, LightGBM often trains faster and scales better with large datasets, though it may require careful tuning to avoid overfitting.
- **Random Forest:** Unlike Random Forest, which builds trees independently, LightGBM builds trees sequentially, focusing on correcting previous errors, typically leading to better predictive performance.

#### Practical Considerations

- **Parameter Tuning:** Key hyperparameters include the number of trees, learning rate, maximum depth, and number of leaves. Proper tuning using cross-validation is crucial.
- **Feature Importance:** LightGBM provides tools to assess feature importance, helping you understand which features contribute most to the predictions.
- **Handling Categorical Variables:** It can handle categorical features directly without needing one-hot encoding, improving both speed and performance.
- **Overfitting:** As with other boosting techniques, overfitting can occur if the model is overly complex. Techniques like early stopping and regularization are important to control this.

LightGBM has become a popular choice in many real-world applications due to its efficiency and effectiveness in handling complex regression problems.


In [8]:
param_grid_lgb = {
    "num_leaves": [31, 50],
    "learning_rate": [0.01, 0.05],
    "n_estimators": [100, 200],
    "reg_alpha": [0, 0.1],
    "reg_lambda": [1, 1.5],
    "subsample": [0.8],
    "colsample_bytree": [0.8]
}

lgb_model = lgb.LGBMRegressor(
    objective='regression',
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

grid_search_lgb = GridSearchCV(
    estimator=lgb_model,
    param_grid=param_grid_lgb,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=0,
    n_jobs=-1
)
grid_search_lgb.fit(X_train, y_train)
print("Best LightGBM parameters:", grid_search_lgb.best_params_)
best_lgb = grid_search_lgb.best_estimator_

holdout_preds_lgb = best_lgb.predict(X_holdout)
rmse_lgb = np.sqrt(mean_squared_error(np.expm1(y_holdout), np.expm1(holdout_preds_lgb)))
print("Holdout RMSE (LightGBM):", rmse_lgb)

Best LightGBM parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'n_estimators': 200, 'num_leaves': 31, 'reg_alpha': 0.1, 'reg_lambda': 1.5, 'subsample': 0.8}
Holdout RMSE (LightGBM): 28462.843752767698


## 5. Refit Models

In [9]:
# Use almost full training data with a small validation split (10%) for early stopping
X_full_train, X_val, y_full_train, y_val = train_test_split(
    train_data, y, test_size=0.1, random_state=42
)

best_xgb.fit(
    X_full_train, y_full_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)

best_lgb.fit(
    X_full_train, y_full_train,
    eval_set=[(X_val, y_val)],
)

# Refit Ridge on the full training data
best_ridge.fit(train_data, y)

Ridge(alpha=10.0)

## 6. Stacking Ensemble

Stacking (or stacked generalization) is an ensemble learning technique where predictions from multiple base models (estimators) are combined using a higher-level model (meta-model) to achieve improved predictive performance.

##### How does stacking work?

1. **Base Models:**  
   First, train several different base models independently (e.g., XGBoost, LightGBM, Ridge regression).

2. **Meta-model:**  
   Then, you use another model (the "meta-model") to learn how to best combine the predictions from the base models. Typically, the meta-model is trained on the predictions of base models using cross-validation to prevent overfitting.

3. **Final Prediction:**  
   Finally, the meta-model makes predictions by combining the base models' outputs, typically achieving better accuracy than any single model alone. (In this case, the base models' outputs and combined using ridge regression.)

In [10]:
# Use our tuned models as base estimators and combine them with a Ridge meta-model
estimators = [
    ('xgb', best_xgb),
    ('ridge', best_ridge),
    ('lgb', best_lgb)
]
meta_model = Ridge(alpha=1.0)
stacking_model = StackingRegressor(
    estimators=estimators,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1
)

# Fit stacking model on the full training data
stacking_model.fit(train_data, y)

StackingRegressor(cv=5,
                  estimators=[('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learnin...
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=200, n_jobs=-1,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                              ('ridge', Ridge(alpha=10.0)),
                              ('lgb',
                               LGBMRegressor(colsample_bytree=0.8,
                                             learning_rate=0.05,
                                             n_estimators=200, n_jobs=-1,
                                             objective='regression',
                                             random_state=42, reg_alpha=0.1,
                                             reg_lambda=1.5, subsample=0.8,
                                             verbose=-1))],
                  final_estimator=Ridge(), n_jobs=-1)

## 7. Predict on Test Set with Fully Trained Model

In [11]:
# Predict on the test set using the stacking ensemble
stacking_preds = stacking_model.predict(test_data)
stacking_preds = np.expm1(stacking_preds)  # Reverse the log transform

## 8. Write to Submission File

In [12]:
submission = pd.DataFrame({
    "Id": test_ID,
    "SalePrice": stacking_preds
})
submission.to_csv("submission.csv", index=False)
print("Submission file created: submission.csv")

Submission file created: submission.csv
